In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from rake_nltk import Rake
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt


In [2]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages

True

In [4]:
df_movies = pd.read_csv("/Users/saishashetty/Downloads/425324-810443-bundle-archive/IMDb movies.csv")
df_ratings = pd.read_csv("/Users/saishashetty/Downloads/425324-810443-bundle-archive/IMDb ratings.csv")
with open('/Users/saishashetty/Downloads/Building-a-Simple-Chatbot-in-Python-using-NLTK-master/chatbot.txt','r', encoding='utf8', errors ='ignore') as fin:
    raw = fin.read().lower()

In [5]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

In [6]:
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [7]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "how are you","hey")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello","Im good", "I am glad! You are talking to me"]

def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [8]:
# Taking a look at all the columns in the dataframe(s)
#print("Columns for Movie Details: \n",df_movies.columns)
#print("Columns for Movie Rating statistics: \n", df_ratings.columns)

# Taking a look at the Movie Details Dataframe
df_movies.head()

# Taking a look at the Movie Rating Statistics Dataframe
df_ratings.head()


,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000574,6.1,537,6.3,6.0,54,17,55,121,122,...,6.0,19.0,6.6,14.0,6.3,64.0,6.0,89.0,6.2,309.0
1,tt0001892,5.9,171,6.1,6.0,5,6,17,41,52,...,5.8,4.0,6.5,8.0,5.9,29.0,6.2,27.0,6.0,114.0
2,tt0002101,5.2,420,5.2,5.0,12,8,16,60,89,...,5.5,14.0,6.2,20.0,4.9,57.0,5.5,197.0,4.7,103.0
3,tt0002130,7.0,2019,6.9,7.0,194,208,386,571,308,...,7.3,74.0,7.4,75.0,7.0,126.0,7.1,452.0,7.0,1076.0
4,tt0002199,5.7,438,5.8,6.0,28,15,42,75,114,...,4.8,10.0,6.5,15.0,5.7,56.0,5.9,161.0,5.7,164.0


In [9]:
# Removing all the unwanted columns from the two Dataframes
df_movies = df_movies[['imdb_title_id','title', 'duration', 'year', 'genre', 'language', 'actors', 'director','description']]
df_ratings = df_ratings[['imdb_title_id', 'mean_vote', 'weighted_average_vote','median_vote', 'total_votes']]

In [10]:
#Again Taking a look at all the columns in the dataframe(s) after dropping unwanted columns
#print("Columns for Movie Details: \n",df_movies.columns)
#print("Columns for Movie Rating statistics: \n", df_ratings.columns)

# Merging the two dataframes and dropping all the nan values
df = pd.merge(df_movies, df_ratings, on='imdb_title_id')
#print("Shape, Before dropping Nan Values: ",df.shape)
df.dropna(inplace = True)
#print("Shape, After dropping Nan Values: ",df.shape)


In [11]:
# Removing all those movies which have a rating of less than 5 and if the number of reviewers are less than 500
df = df[(df['mean_vote'] >= 5) & (df['total_votes'] >= 500)]
#print("Shape of the dataframe after dropping selected movies: ",df.shape)

# Selecting only the movies available in English, Tamil, Hindi, 

# When you are running code in your system
# comment this line below and run it
#df = df[df['language'].str.contains(r'Tamil|Kannada|Telugu|Hindi|Malayalam')]


In [12]:
df.reset_index(drop=True,inplace=True)
df = df.apply(lambda x: x.astype(str).str.lower())

df['Key_words'] = ''
r = Rake()
for index, row in df.iterrows():
    r.extract_keywords_from_text(row['description'])
    key_words_dict_scores = r.get_word_degrees()
    row['Key_words'] = list(key_words_dict_scores.keys())
    df['Key_words'][index] = row['Key_words']

In [13]:
df['genre'] = df['genre'].map(lambda x: x.split(','))
df['actors'] = df['actors'].map(lambda x: x.split(',')[:3])
df['director'] = df['director'].map(lambda x: x.split(','))
for index, row in df.iterrows():
    row['genre'] = [x.lower().replace(' ','') for x in row['genre']]
    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
    row['director'] = [x.lower().replace(' ','') for x in row['director']]


In [14]:
df['Bag_of_words'] = ''
columns = ['genre', 'director', 'actors', 'language', 'Key_words']
for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words
    df['Bag_of_words'][index] = words

In [15]:
df3 = df[['title','Bag_of_words']]

count = CountVectorizer()
count_matrix = count.fit_transform(df3['Bag_of_words'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(df['title'])


In [16]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [17]:
def recommend(title, num=10, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    print(idx)
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:num].index)

    for i in top_10_indices:
        recommended_movies.append(list(df['title'])[i])

    return recommended_movies

In [20]:
flag=True
print("ROBO: My name is Robo. Type 'help' for guidance, 'review' to review a movie and let others know your opinion, 'recommend' so we can recommend you a movie based on your preference :)")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='recommend'):
        if(user_response!='review'):
            if(user_response=='thanks' or user_response=='thank you' ):
                flag=False
                print("ROBO: You are welcome..")
            else:
                if(greeting(user_response)!=None):
                    print("ROBO: "+greeting(user_response))
                else:
                    print("ROBO: ",end="")
                    print(response(user_response))
                    sent_tokens.remove(user_response)
        else:
            flag=False
            print("ROBO: Alright...")  
    else:
        flag=False
        print("ROBO: Alright...")      
        
string = input("Enter a movie based on which we could recommend one to you: ")
#num = int(input("How many such similar movies do you want?"))
print("Similar movies are")
string = string.lower()
recommend(string)
        

ROBO: My name is Robo. Type 'help' for guidance, 'review' to review a movie and let others know your opinion, 'recommend' so we can recommend you a movie based on your preference :)
help
ROBO: i am here to help you, you can either review a movie or you can get recommendations based on your choice
recommend
ROBO: Alright...
Enter a movie based on which we could recommend one to you: king kong
Similar movies are
543


['the son of kong',
 'the most dangerous game',
 'the last days of pompeii',
 'c.h.u.d.',
 'the amazing spider-man 2',
 'war of the worlds',
 'mad doctor of blood island',
 '2019 - dopo la caduta di new york',
 'vamps']